# Web Scrapping Surfline.com

## Importaciones

In [56]:
from datetime import datetime
import time
from bs4 import BeautifulSoup
import os
from selenium import webdriver
import pandas as pd



## Analisis inicial

In [ ]:
'''
# Configurar Selenium con el controlador del navegador
driver = webdriver.Chrome()  # Asegúrate de que el controlador esté instalado y en tu PATH
url = "https://www.surfline.com/surf-report/playa-grande/5842041f4e65fad6a7708ce3?view=table"
driver.get(url)

# Esperar a que la página cargue completamente
time.sleep(5)  # Ajusta el tiempo según la velocidad del sitio

# Obtener el HTML generado
html = driver.page_source

# Analizar con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extraer tablas
tables = soup.find_all('table')
for i, table in enumerate(tables):
    print(f"\nTabla {i + 1}:")
    rows = table.find_all('tr')
    for j, row in enumerate(rows):
        print(f"  - Fila {j + 1}:")
        cells = row.find_all('td')
        for k, cell in enumerate(cells):
            print(f"    - Celda {k + 1}: {cell.get_text(strip=True)}")

# Cerrar el navegador
driver.quit()'''



Tabla 1:
  - Fila 1:

Tabla 2:
  - Fila 1:
    - Celda 1: 6am
    - Celda 2: 
  - Fila 2:
    - Celda 1: Noon
    - Celda 2: 
  - Fila 3:
    - Celda 1: 6pm
    - Celda 2: 

Tabla 3:
  - Fila 1:
    - Celda 1: 0.6-0.9
    - Celda 2: POOR
    - Celda 3: 0.8m7s
    - Celda 4: 0.1m12s
    - Celda 5: 0.1m9s
    - Celda 6: 2442kph
  - Fila 2:
    - Celda 1: 0.3-0.6
    - Celda 2: POOR
    - Celda 3: 0.6m7s
    - Celda 4: 0.1m11s
    - Celda 5: 0.1m10s
    - Celda 6: 2435kph
  - Fila 3:
    - Celda 1: 0.3-0.6
    - Celda 2: POOR
    - Celda 3: 0.4m7s
    - Celda 4: 0.1m11s
    - Celda 5: 0.1m10s
    - Celda 6: 2228kph

Tabla 4:
  - Fila 1:
    - Celda 1: 15ºc
    - Celda 2: 1009mb
    - Celda 3: 100%
  - Fila 2:
    - Celda 1: 16ºc
    - Celda 2: 1004mb
    - Celda 3: 100%
  - Fila 3:
    - Celda 1: 17ºc
    - Celda 2: 1000mb
    - Celda 3: 100%

Tabla 5:
  - Fila 1:
    - Celda 1: 6am
    - Celda 2: 
  - Fila 2:
    - Celda 1: Noon
    - Celda 2: 
  - Fila 3:
    - Celda 1: 6pm
    - Celda

Llego a la conclusión de que hay 16 tablas finalmente, por lo cual voy a comentar el código para no volver a realizarlo
Debajo realizo una mejora en el código para seleccionar únicamente las tablas requeridas.

In [ ]:
# Configurar Selenium con el controlador del navegador
driver = webdriver.Chrome()  # Asegúrate de tener el controlador instalado y configurado
url = "https://www.surfline.com/surf-report/playa-grande/5842041f4e65fad6a7708ce3?view=table"
driver.get(url)

# Esperar a que la página cargue completamente
time.sleep(5)  # Ajusta el tiempo según la velocidad del sitio

# Obtener el HTML generado
html = driver.page_source

# Analizar con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extraer tablas relevantes (Tabla 2, Tabla 3, Tabla 4)
tables = soup.find_all('table')

# Procesar tabla 2 (Horas del día)
table_2 = tables[1]
rows_table_2 = table_2.find_all('tr')
times = [row.find_all('td')[0].get_text(strip=True) for row in rows_table_2]

# Procesar tabla 3 (Información de olas y viento)
table_3 = tables[2]
rows_table_3 = table_3.find_all('tr')
data_table_3 = []
for row in rows_table_3:
    cells = row.find_all('td')
    data_row = [cell.get_text(strip=True) for cell in cells]
    
    # Corregir formato para las columnas de Swell y Wind
    for idx in [2, 3, 4]:  # Índices de Primary Swell, Secondary Swell 1, Secondary Swell 2
        if idx < len(data_row):
            data_row[idx] = data_row[idx].replace("m", "m ").replace("s", "s")
    data_row[5] = data_row[5].replace("kph", " kph")  # Corregir formato del viento
    
    data_table_3.append(data_row)

# Procesar tabla 4 (Información meteorológica)
table_4 = tables[3]
rows_table_4 = table_4.find_all('tr')
data_table_4 = []
for row in rows_table_4:
    cells = row.find_all('td')
    data_table_4.append([cell.get_text(strip=True) for cell in cells])


In [54]:
print(f'{times}\n{"*"*50}\n{data_table_3}\n{"*"*50}\n{data_table_4}')

['6am', 'Noon', '6pm']
**************************************************
[['0.6-0.9', 'POOR', '0.8m 7s', '0.1m 12s', '0.1m 9s', '2442 kph'], ['0.3-0.6', 'POOR', '0.6m 7s', '0.1m 11s', '0.1m 10s', '2435 kph'], ['0.3-0.6', 'POOR', '0.4m 7s', '0.1m 11s', '0.1m 10s', '2228 kph']]
**************************************************
[['15ºc', '1009mb', '100%'], ['16ºc', '1004mb', '100%'], ['17ºc', '1000mb', '100%']]


Ordeno finalmente el df según los datos que necesito y de la manera que los necesito

In [55]:
# Obtener la fecha de hoy
today_date = datetime.today().strftime('%Y-%m-%d')

# Combinar los datos en un DataFrame
combined_data = []
for i in range(len(times)):
    combined_data.append({
        "Date": today_date,  # Agregar columna de fecha
        "Time": times[i],
        "Surf(m)": data_table_3[i][0],  # Primera columna de la tabla 3
        "Rating": data_table_3[i][1],  # Segunda columna de la tabla 3
        "Primary Swell": data_table_3[i][2],  # Tercera columna de la tabla 3
        "Secondary Swell": f"{data_table_3[i][3]} | {data_table_3[i][4]}",  # Combina Swell 2 y Swell 3
        "Wind": data_table_3[i][5],  # Sexta columna de la tabla 3
        "Temperature": data_table_4[i][0],  # Primera columna de la tabla 4
        "Pressure": data_table_4[i][1],  # Segunda columna de la tabla 4
        "Probability": data_table_4[i][2],  # Tercera columna de la tabla 4
    })

df_scrap = pd.DataFrame(combined_data)

In [ ]:

# Definir la ruta completa al archivo Excel
file_path = "C:\\Users\\dalarroy\\surf-prediction-project\\data\\pg_scrap_surfline.xlsx"

# Leer el archivo Excel si existe
if os.path.exists(file_path):
    # Cargar los datos existentes
    existing_df = pd.read_excel(file_path, engine='openpyxl')
    
    # Eliminar las filas duplicadas basándose solo en la columna "Date" para evitar añadir las mismas filas
    df_scrap = pd.concat([existing_df, df_scrap]).drop_duplicates(subset=["Date"], keep="last").reset_index(drop=True)
else:
    # Si el archivo no existe, solo usa los datos nuevos
    df_scrap = df_scrap.reset_index(drop=True)

# Guardar el DataFrame actualizado en el archivo Excel
df_scrap.to_excel(file_path, index=False, engine='openpyxl')

print(f"El DataFrame se ha guardado correctamente en '{file_path}'")


El DataFrame se ha guardado correctamente en 'C:\Users\dalarroy\surf-prediction-project\data\pg_scrap_surfline.xlsx'
